In [151]:
from mongoengine import *
import numpy as np
import datetime 
import pandas
import matplotlib.pyplot as plt
from sklearn import *

In [152]:
connect("GDP-test", host = "localhost", port = 27017)
class Detection(Document):
    id_webcam = IntField(required=True)
    city = StringField(required=True)
    location = StringField(required=True)
    latitude = FloatField(required=True)
    longitude = FloatField(required=True)
    numPeople = IntField(required=True)
    date = DateTimeField(required=True)
    time = DateTimeField(required=True)
    type = IntField(required=True)
    weather_description = StringField()
    temperature = FloatField()
    day_of_week = IntField()

In [199]:
print(Detection.objects.distinct('city')) # vedere se posso prendere solo le città per poi fare query distinte
table = pandas.DataFrame(Detection.objects(city='Roma').as_pymongo()) #prelevo i dati di roma
table = table.drop(columns = ['_id','id_webcam','city','date','type','location','latitude','longitude']) #tolgo colonne inutili
table['time'] = table['time'].dt.strftime('%H').astype(int)#modifico le colonne con i valori che mi servono
table['weather_description'] = pandas.get_dummies(table['weather_description']) #il weather viene convertito con parametri interi

#divide the dataframe 70-30 for train and test
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    table[['time','temperature','day_of_week','weather_description']], 
    table['numPeople'], 
    test_size = 0.33, shuffle = True, random_state= 42)


['Roma', 'Novska', 'Djakovo', 'Krk']


In [200]:
model = sklearn.linear_model.LinearRegression()
#provo a dargli in pasto tutto
#prima di questo ora bisogna dividere tutto il dataset in 70-30
model.fit(x_train, y_train)


# The coefficients
print('Coefficients: \n', model.coef_)

# The mean square error
print("Residual sum of squares: %.2f" % np.mean((model.predict(x_test) - y_test) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % model.score(x_test, y_test))



Coefficients: 
 [-0.0417654   0.94917226 -0.39676953  0.82073259]
Residual sum of squares: 8.27
Variance score: 0.57


In [87]:

#robe vecchie sbagliate non guardare

time, temperature, weather, numP = table['time'].dt.strftime('%H').astype(int),table['temperature'],table['weather_description'],table['numPeople']
time = np.asarray(time)
temperature = np.asarray(temperature)
weather = np.asarray(weather)
numP = np.asarray(numP)
weatherDict = {'clear sky':0}

In [60]:
for i in range(weather.size):
    weather[i] = weatherDict[weather[i]]

In [61]:
temperature = temperature.reshape(-1,1)
time = time.reshape(-1,1)
weather = weather.reshape(-1,1)
timeTemp = np.concatenate((time,temperature,weather),axis = 1)
print(timeTemp)
model = sklearn.linear_model.LinearRegression()
model.fit(timeTemp,numP) 

[[11 13.48 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]]


LinearRegression()